# Don't run this on Colab! Our file size is too large.

In [8]:
import pandas as pd
from pandas import json_normalize
import json
import requests

# 1. Call the API to get historical data

In [9]:
ls = []

date_interval = pd.date_range(start="2021/03/10", end = "2021/06/17", freq="15t")

converted_dates=[]

for date in date_interval:
  converted_dates.append(date.strftime("%Y%m%d-%H%M"))
for date1 in converted_dates:
  print(date1)
  url = "https://api.data.gov.hk/v1/historical-archive/get-file?url=https%3A%2F%2Fresource.data.one.gov.hk%2Ftd%2Fcarpark%2Fvacancy_all.json&time=" + date1
  vacancy_df = json.loads(requests.get(url).content)
  ls.append(vacancy_df)

## Save all the dates we call to the API

In [13]:
pd.DataFrame(converted_dates).to_csv('date.csv', index=False)

## Save the historical data as raw.csv

In [40]:
copy = ls

res = []
for i in copy:
    if i not in res:
        res.append(i)
        
df = pd.DataFrame(res)
df.to_csv('raw.csv', index=False)

# 2. Call the API to get basic info

In [6]:
url = 'https://resource.data.one.gov.hk/td/carpark/basic_info_all.json'

d = json.loads(requests.get(url).content)
d = list(d.values())[0]

## Save the historical data as car_park_info.csv

In [7]:
df = json_normalize(d)
df = df.drop(['website_en', 'website_tc', 'website_sc'], axis=1).set_index('park_id')
df.to_csv('car_park_info.csv')